In [ ]:
using Random
using Distributions
using Statistics
using Printf
using Plots

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

In [ ]:
λ = 0.5; # birth-death rates
μ = 1.;
ρ = λ/μ;
q = λ + μ;

X0 = 0; # initial population
T0 = 0.;
Tfinite = 10;
N = 2;

K = 10^4;

Tmax = 10^6;

# Finite Time Horizion Problem
Compute 
$$
\mathbb{P}(X_T>N)
$$
with $T$ = 10 and $N = 5$.  This is accomplished by running $K$ independent trials of the M/M/1 process.

In [ ]:
X_samples = zeros(K);
Random.seed!(100);

for k in 1:K
    # run each independent trial
    T = 0.;
    X = X0;
    while(T< Tfinite)
        if(X > 0 )
            ΔT = rand(Exponential(1/q));
            ΔX = 2*rand(Categorical([μ/q, λ/q]))-3;
            
        else
            ΔT = rand(Exponential(1/λ));
            ΔX = 1;
        end
        if(T+ΔT< Tfinite)
            X += ΔX
        end
        T += ΔT;
    end
    X_samples[k] = X;    
end

p_est = mean(X_samples .> N);
@show p_est;

# Steady State Simulation
Estimate
$$
\mathbb{E}[X]
$$
for the M/M/1 queue.  This is estimated with
$$
\frac{1}{T}\int_{0}^T X_t dt = \frac{1}{T}\sum_n Y_{n} (T_{n+1} - T_n)
$$

In [ ]:
X = X0;
T = 0.;
X_trajectory = Int[X];
ΔX_vals = [];
T_trajectory = Float64[T];

Random.seed!(100);

while(T<Tmax)
    # compute holding time
    if(X > 0 )
        ΔT = rand(Exponential(1/q));
        ΔX = 2*rand(Categorical([μ/q, λ/q]))-3;

    else
        ΔT = rand(Exponential(1/λ));
        ΔX = 1;
    end
    X += ΔX;
    T += ΔT;
    push!(X_trajectory, X);
    push!(T_trajectory, T);     
    # end
end


In [ ]:
@show sum(X_trajectory[1:end-1] .* diff(T_trajectory))/T;
EX = ρ/(1-ρ);
@show EX;

In [ ]:
Xavg_trajectory = cumsum(X_trajectory[1:end-1] .* diff(T_trajectory))./(T_trajectory[2:end]);

In [ ]:
plot(T_trajectory[1:end-1], Xavg_trajectory,label="Running Avg.", legend=:bottomright)
xlabel!("t")